## Plots

* Plotting is typically done through the `Plots` package.

* Plotting in Julia is relatively immature: it requires a backend.

* There is a high time-to-first plot time but it is smooth from there.

## Plotting grammar.
* Plotting follows a predictable grammar through the `plot` API call.

* Plots are arranged in x data, y data formats. Series are delineated with spaces.

* Optional keywords denote styling and can be shorted:
	1) seriestype/st = {"scatter", "line", "contour", etc.}
	2) color/c = {:red, RGB(0,1,0), etc} (can be input as a list)
	3) label = "series label"
	4) xlabel= , title= , ylabel=


In [ ]:
#| echo: true
using Plots
pyplot()
x = collect(0:0.1:10)
y = sin.(x)
plot(x, y; st="line", label=false, title="Plot of a sin wave", ylabel="sin", xlabel="x", colour=:blue)

## Plot Objects

* Plots can be attached to objects; last plot is attached to an internal object.

* Plot objects can be added to with the `plot!(plot_object, x,y...; keywords=...) call.

* Plot objects can be referenced later.


In [ ]:
#| echo: true
x = 0:0.01:2pi
y1 = sin.(x)
y2 = cos.(x)
y3 = tan.(x)

trig_plot = plot(x, [y1, y2]; title="Trigonometric Plots", xlabel="x", label=["sin(x) cos(x)"])
plot!(trig_plot, x, y3; label="tan(x)")

## Plot Layouts

*Subplots are handled through layout specification.

*Simple layouts can specify a grid format. A more complicated layout can be defined with the `@layout`

*A layout is arranged as a matrix with `a,b,c` denoting the plots. They are optionally given widths and heights e.g. `a{0.2h 0.3w}`.


In [ ]:
#| echo: true
p1 = plot(rand(10)); p2 = plot(rand(10)); p3 = plot(rand(10)); p4 = plot(rand(10));

# plot(p1, p2, p3, p4; layout=(2,2))

L = @layout [a{0.2h, 0.4w} b; d{0.8h} d{0.5w}]
plot(p1, p2, p3, p4; layout=L)

## StatsPlots

* Plots supports recipes and thus a Plotting ecosystem.

* StatsPlots allows for efficient DataFrames plotting.

* A slightly more complex grammar; documentation found [here](https://docs.juliaplots.org/latest/generated/statsplots/)


In [ ]:
#| echo: true
using DataFrames, CSV, StatsPlots

experiment = CSV.File("plotting_data.csv") |> DataFrame
@df experiment density(:BodyLength, group = (:Planet, :Diet), legend=:topright)

## Save Plots

* Plots are saved through the `savefig` API call.

* It specifies the path and plot object to be saved.

* The path defines the file type.


In [ ]:
#| echo: true
plt = plot(collect(1:10), rand(10); st=:scatter)
savefig(plt, "my_first_plot.png")

## Animate

* A useful tool to visualise data is animation.

* This is done through the @animate macro applied to a `for` loop.

* The animation can be saved through the `gif` API call. This takes the animation object, then path, and optional FPS.


In [ ]:
#| echo: true
x = collect(0:0.05:2pi)
y = sin.(x)
anim_obj = @animate for t in 1:length(x)
	plot(x[1:t], y[1:t]; xrange=(0,2pi), legend=false, xlabel="x", ylabel="sin(x)", yticks=false)
end
gif(anim_obj, "my_first_animation.gif", fps=30)

## Performance Computing

* Efficient and fast code is one of the **big** draw cards of Julia.

* It *usually* comes for free, but not always.

* There are some common 'tricks' to employ to improve code efficiency and speed.

## Data Access Patterns

* Data in Julia is organised in a *column-major* format.

* Data is laid out with the columns stacked end-to-end. The fastest way to access them is through the *rows*. 

* Slow code uses columns as the 'fast-changing' index, but is more traditional. When looping use columns as the outer variable.


In [ ]:
#| echo: true
L = 10^7
a1 = rand(L, L); a2 = a1'; v1 = zeros(L); v2 = zeros(L);

@time for j = 1:size(a)[2]
	v1[j] = sum(a1[:,j])
end

@time for j = size(a2)[1]
	v2[j] = sum(a2[i, :])
end

v1 == v2

::: {.notes}

* This is common for scientific languages (R, MatLab, Mathematica...), not for traditional languages (C, Python...).

:::

## @inbounds

* Bounds checking is a useful operation in an interpreted language throwing an error if accessing an invalid index.

* It is expensive and fast languages like C don't use it - at user peril!

* It is activated with the `@inbounds` macro and distributes to all nested loops.


In [ ]:
#| echo: true
v = rand(10^9); res = 0
for i = 1:length(v)
	res += v[1]
end

#| echo: true
v = rand(10^9); res = 0;
@inbounds for i = 1:(length(v) + 1)
	res += v[1]
end

#| echo: true
v = rand(10^9); res = 0;
@time for i = 1:length(v)
	res += v[1]
end

#| echo: true
v = rand(10^9); res = 0;
@time @inbounds for i = 1:length(v)
	res += v[1]
end
::: {.notes}

* It is also safe, so you don't make numerical errors when it accesses (valid) but not in-range data. It's a hard error to track.

* When developing, make sure code works and when it needs to be scaled turn it on.

:::

## @SIMD

* SIMD stands for single instruction multiple data and is a machine level optimisation in modern CPUs.

* It allows many mathematical operations to be vectorised and optimised within the CPU cycle.

* Turn it on using the ``@simd` macro. For well organised data you can expect some speed up.

#| echo: true
v = rand(10^9)
@time for i = 1:length(v)
	v[i] += 1
end

@time @simd for i = 1:length(v)
	v[i] -= 1
end

## Type Mutation

* Julia has excellent typing and type inference but abstraction puts demands on the compiler.

* When types mutate the compiler works hard to "deal with it" which is good for the end user.

* To avoid this Julia can strictly type variables and functions which alleviates the pressure.

#| echo: true
a::Array{Float32, 1} = [4.0, 2.0, 1.2] # a strictly typed vector.
f(x::Array{Float32, 1}) = sum(x) # a strictly typed function.

## High Performance Computing

* HPC refers to distributing compute tasks in an efficient way.

* It typically refers to parallel computing which comes in two general flavours: mutliple CPU threads, or GPUs.

* These follow similar principles but the architectures require different coding styles.

* Julia abstracts many of the 'gotchas'.

## Distributed

* Parallel computing with multiple threads is available through the `Distributed` package.

* It supports low level (``spawn``, ``fetch``, ``remotecall`` etc.) methods.

* More often it is used to parallelise loops through the ``@distributed`` macro before a `for` loop block.

* To make processes available use the ``addprocs(n)`` method.

In [ ]:
#| echo: true
using Distributed
addprocs(4)

@time for i = 1:10^9
	i^2
end

@time @distributed for i = 1:10^9
	i^2
end

## Distributed Reduce

* A useful operation for parallesiation is reduction.

* A distributed for block can specify a binary operation to reduce on.


In [ ]:
#| echo: true
sumodds = @distribute (+) for i = 1:10000
	Int(isodd(i))
end

## Shared Functions

* Each thread has access to its own local environment and thus function definitions.

* The `@everywhere` macro is used to specify that a function/package can be accessed from thread.


In [ ]:
#| echo: true
@everywhere function myfunc(x)
	return x^2 - y^3 + sin(x * y)
end

@everwhere using Statistics

## Shared Data

* Additional threads also do not have access to data on the master thread.

* The `SharedArrays` package allows for memory to be shared between threads through a `SharedArray` object.


In [ ]:
#| echo: true
@everywhere using SharedArrays

v = SharedArray(zeros(5))

@distribute for i = 1:length(v)
	v[i] = i^2
end
println(v) 

## Race Conditions and Synchronisation

* Data and thread access is typically asynchronous.

* What happens when two threads depend on each other catastrophically?

* What happens when two threads try to access/modify data at the same time.

* These are *race* conditions and the second can be dealt with using *atomic* operations and synchronisation.


## pmap

* Functional progamming styles are supported through `pmap` which behaves like `map`.

* There is a slight performance cost and is best used with complex function calls.


In [ ]:
#| echo: true
pmap( x -> x^2, 1:5)

## GPU Compute

* Graphical Processing Units use a special hardware layout to launch hundreds of threads with a low clock speed.

* They are slow at performing individual tasks, but can do many simultaneously. 

* The startup time is slow but for large jobs speed ups of 20x to 100x are commonplace.

* Julia supports them through a generic `GPUArrays` backend, but users interface with a card specific API: `AMDGPUs`, `CUDA`, `Metal`.

* They are best used for linear algebra and vectorisable tasks.

## GPU Basics

* APIs have a shared array data structure which can load CPUs to the GPU: `CUDA.CuArray`, `AMDGPU.ROCArray`, `Metal.MtlArray`.

* Common functions are overloaded to these data structures: `sum, +, -, *, /,  ^, ., sin, cos, exp` etc.

* Functional programming is supported through `map` and `reduce` frameworks.

* Indexed functions are *highly* discouraged: the GPU has to fall back on the CPU incurring a *very* high cost.

* Calls are asyncronous and the `PACKAGE.@sync` macro will synchronise data which is important for data dependencies.

In [ ]:
using CUDA

a = rand(10^9);
a_gpu = CuArray(a);

@time sum(a);
CUDA.@time CUDA.@sync sum(a_gpu)

## Kernels

* GPU calls are handled by `kernels`.

* These are normally written in the backend before being overloaded into existing functions.

* They use data arranged in Threads, and Blocks, accessing indexes asynchronously by a function (ThreadId, BlockId) -> idx.

* When the threads synchronise the data in memory is modified and the computation moves to the next block.

* Custom kernels may be written and launched with the `@cuda` kernel. 

* The asynchronous programming style is different to a traditional style and beyond the scope of this course.